# Predicting Teacher Turnover

__Meaghan Ross__

Flatiron School Capstone

### Business Understanding

### Data Understanding

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

### Data Preparation

In [2]:
ls data/

employee_information_Nov2014.csv  employee_information_Oct2018.csv
employee_information_Oct2015.csv  employee_information_Oct2019.csv
employee_information_Oct2016.csv  employee_information_Oct2020.csv
employee_information_Oct2017.csv


In [36]:
df_2015=pd.read_csv('data/employee_information_April2015.csv')
df_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19138 entries, 0 to 19137
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      19137 non-null  object
 1   FIRST_NAME                     19138 non-null  object
 2   PAY_RATE_TYPE                  19138 non-null  object
 3   PAY_RATE                       19138 non-null  int64 
 4   TITLE_DESCRIPTION              19138 non-null  object
 5   HOME_ORGANIZATION              19138 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  19138 non-null  object
 7   ORGANIZATION_LEVEL             19138 non-null  object
 8   TYPE_OF_REPRESENTATION         19138 non-null  object
 9   GENDER                         19138 non-null  object
 10  RUN_DATE                       19138 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.6+ MB


In [37]:
df_2016=pd.read_csv('data/employee_information_April2016.csv')
df_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      17561 non-null  object
 1   FIRST_NAME                     17562 non-null  object
 2   PAY_RATE_TYPE                  17562 non-null  object
 3   PAY_RATE                       17562 non-null  int64 
 4   TITLE_DESCRIPTION              17562 non-null  object
 5   HOME_ORGANIZATION              17562 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  17562 non-null  object
 7   ORGANIZATION_LEVEL             17562 non-null  object
 8   TYPE_OF_REPRESENTATION         17561 non-null  object
 9   GENDER                         17562 non-null  object
 10  RUN_DATE                       17562 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.5+ MB


In [39]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23363 entries, 0 to 23362
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           23362 non-null  object 
 1   FIRST_NAME                          23363 non-null  object 
 2   PAY_RATE_TYPE_2015                  19356 non-null  object 
 3   PAY_RATE_2015                       19356 non-null  float64
 4   TITLE_DESCRIPTION_2015              19356 non-null  object 
 5   HOME_ORGANIZATION                   23363 non-null  object 
 6   HOME_ORGANIZATION_DESCRIPTION_2015  19356 non-null  object 
 7   ORGANIZATION_LEVEL_2015             19356 non-null  object 
 8   TYPE_OF_REPRESENTATION_2015         19356 non-null  object 
 9   GENDER_2015                         19356 non-null  object 
 10  RUN_DATE_2015                       19356 non-null  object 
 11  PAY_RATE_TYPE_2016                  17861

In [40]:
df_merged['TITLE_DESCRIPTION_2016'].value_counts()

TEACHER,FULL TIME                6652
TEACHER,SPEC EDUCATION           1314
ONE TO ONE ASST, SPECIAL ED       764
CLASSROOM ASST,SP ED,SV HND       596
GENERAL CLEANER, 8 HOURS          503
                                 ... 
SPECIAL ED MEDICAL CONSULTANT       1
DEPUTY CHIEF INFO OFFICER           1
CHIEF FINANCIAL OFFICER             1
REGIONAL ACCESS REP II              1
BENEFITS MANAGER                    1
Name: TITLE_DESCRIPTION_2016, Length: 552, dtype: int64

In [41]:
df_2017 = pd.read_csv('data/employee_information_April2017.csv')
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18353 entries, 0 to 18352
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      18352 non-null  object
 1   FIRST_NAME                     18353 non-null  object
 2   PAY_RATE_TYPE                  18353 non-null  object
 3   PAY_RATE                       18353 non-null  int64 
 4   TITLE_DESCRIPTION              18353 non-null  object
 5   HOME_ORGANIZATION              18353 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  18353 non-null  object
 7   ORGANIZATION_LEVEL             18353 non-null  object
 8   TYPE_OF_REPRESENTATION         18343 non-null  object
 9   GENDER                         18353 non-null  object
 10  RUN_DATE                       18353 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.5+ MB


In [42]:
df_2018 = pd.read_csv('data/employee_information_April2018.csv')
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19181 entries, 0 to 19180
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      19180 non-null  object
 1   FIRST_NAME                     19181 non-null  object
 2   PAY_RATE_TYPE                  19181 non-null  object
 3   PAY_RATE                       19181 non-null  int64 
 4   TITLE_DESCRIPTION              19181 non-null  object
 5   HOME_ORGANIZATION              19181 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  19181 non-null  object
 7   ORGANIZATION_LEVEL             19181 non-null  object
 8   TYPE_OF_REPRESENTATION         18864 non-null  object
 9   GENDER                         19181 non-null  object
 10  RUN_DATE                       19181 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.6+ MB


In [43]:
df_2019 = pd.read_csv('data/employee_information_April2019.csv')
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20151 entries, 0 to 20150
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LAST_NAME                      20149 non-null  object
 1   FIRST_NAME                     20151 non-null  object
 2   PAY_RATE_TYPE                  20151 non-null  object
 3   PAY_RATE                       20151 non-null  int64 
 4   TITLE_DESCRIPTION              20151 non-null  object
 5   HOME_ORGANIZATION              20151 non-null  object
 6   HOME_ORGANIZATION_DESCRIPTION  20151 non-null  object
 7   ORGANIZATION_LEVEL             20151 non-null  object
 8   TYPE_OF_REPRESENTATION         20151 non-null  object
 9   GENDER                         20151 non-null  object
 10  RUN_DATE                       20151 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.7+ MB


In [44]:
df_merged_2016 = pd.merge(df_2015, df_2016, 
                          on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                          how='outer', suffixes=('_2015', '_2016'))

In [45]:
df_merged_2017 = pd.merge(df_merged_2016, df_2017, 
                          on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                          how='outer', suffixes=(' ', '_2017'))

In [47]:
df_merged_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29015 entries, 0 to 29014
Data columns (total 27 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           29014 non-null  object 
 1   FIRST_NAME                          29015 non-null  object 
 2   PAY_RATE_TYPE_2015                  19623 non-null  object 
 3   PAY_RATE_2015                       19623 non-null  float64
 4   TITLE_DESCRIPTION_2015              19623 non-null  object 
 5   HOME_ORGANIZATION                   29015 non-null  object 
 6   HOME_ORGANIZATION_DESCRIPTION_2015  19623 non-null  object 
 7   ORGANIZATION_LEVEL_2015             19623 non-null  object 
 8   TYPE_OF_REPRESENTATION_2015         19623 non-null  object 
 9   GENDER_2015                         19623 non-null  object 
 10  RUN_DATE_2015                       19623 non-null  object 
 11  PAY_RATE_TYPE_2016                  18144

In [48]:
df_merged_2018 = pd.merge(df_merged_2017, df_2018, 
                          on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                          how='outer', suffixes=('_2017', '_2018'))

In [49]:
df_merged_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34444 entries, 0 to 34443
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           34443 non-null  object 
 1   FIRST_NAME                          34444 non-null  object 
 2   PAY_RATE_TYPE_2015                  19960 non-null  object 
 3   PAY_RATE_2015                       19960 non-null  float64
 4   TITLE_DESCRIPTION_2015              19960 non-null  object 
 5   HOME_ORGANIZATION                   34444 non-null  object 
 6   HOME_ORGANIZATION_DESCRIPTION_2015  19960 non-null  object 
 7   ORGANIZATION_LEVEL_2015             19960 non-null  object 
 8   TYPE_OF_REPRESENTATION_2015         19960 non-null  object 
 9   GENDER_2015                         19960 non-null  object 
 10  RUN_DATE_2015                       19960 non-null  object 
 11  PAY_RATE_TYPE_2016                  18489

In [50]:
df_merged_2019 = pd.merge(df_merged_2018, df_2019, 
                          on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                          how='outer')

In [68]:
df_merged_2019['TITLE_DESCRIPTION_2015'].value_counts()

TEACHER,FULL TIME                6709
TEACHER,SPEC EDUCATION           1335
NOON TIME AIDE, 3 HRS             850
NOON TIME AIDE, 4 HRS             826
ONE TO ONE ASST, SPECIAL ED       691
                                 ... 
DIRECTOR,TRANSLATION SERVICES       1
ASST DIR,EMPLOYEE BENEFITS          1
SUPERINTENDENT OF SCHOOLS           1
LEAD AUDIT CLRK                     1
STRATEGY ANALYST I                  1
Name: TITLE_DESCRIPTION_2015, Length: 541, dtype: int64

In [80]:
df_merged_2019.loc[(df_merged_2019['PAY_RATE_2015'].isna())&(df_merged_2019['TITLE_DESCRIPTION_2016']=='TEACHER,FULL TIME')]

,LAST_NAME,FIRST_NAME,PAY_RATE_TYPE_2015,PAY_RATE_2015,TITLE_DESCRIPTION_2015,HOME_ORGANIZATION,HOME_ORGANIZATION_DESCRIPTION_2015,ORGANIZATION_LEVEL_2015,TYPE_OF_REPRESENTATION_2015,GENDER_2015,...,GENDER_2018,RUN_DATE_2018,PAY_RATE_TYPE,PAY_RATE,TITLE_DESCRIPTION,HOME_ORGANIZATION_DESCRIPTION,ORGANIZATION_LEVEL,TYPE_OF_REPRESENTATION,GENDER,RUN_DATE
20523,ABDALLAH,JUWAYRIYAH,NaN,NaN,NaN,1470,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20532,ADAIR,DARRELL,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,...,M,4/1/2018,SALARIED,67706.0,"TEACHER,FULL TIME","FRANKLIN, BENJAMIN HIGH SCHOOL",HIGH SCHOOL,PFT-TEACHER,M,4/1/2019
20533,ADAMS,CA-TISHA,NaN,NaN,NaN,4394,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20536,ADAMS,KATHY,NaN,NaN,NaN,7220,NaN,NaN,NaN,NaN,...,F,4/1/2018,SALARIED,76461.0,"TEACHER,FULL TIME","CARNELL, LAURA H. SCHOOL",ELEMENTARY SCHOOL,PFT-TEACHER,F,4/1/2019
20537,ADAMS,LYNDA,NaN,NaN,NaN,8410,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24560,ZIEVE,HANNAH,NaN,NaN,NaN,6460,NaN,NaN,NaN,NaN,...,F,4/1/2018,SALARIED,54364.0,"TEACHER,FULL TIME",HILL-FREEDMAN WORLD ACADEMY,HIGH SCHOOL,PFT-TEACHER,F,4/1/2019
24561,ZIMMER,ASHLEY,NaN,NaN,NaN,2210,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24563,ZOTTER,JORDAN,NaN,NaN,NaN,5320,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24567,ZWOLAK,PAUL,NaN,NaN,NaN,2290,NaN,NaN,NaN,NaN,...,M,4/1/2018,SALARIED,69207.0,"TEACHER,FULL TIME",FRANKLIN LEARNING CENTER,HIGH SCHOOL,PFT-TEACHER,M,4/1/2019


In [82]:
ls data/

SPR_SY1415_School_Metric_Scores_20160208.xlsx
SPR_SY1516_School_Metric_Scores_20170203.xlsx
SPR_SY1617_School_Metric_Scores_20180206.xlsx
SPR_SY1718_School_Metric_Scores_20190129.xlsx
SPR_SY1819_School_Metric_Scores_20200127.xlsx
employee_information_April2015.csv
employee_information_April2016.csv
employee_information_April2017.csv
employee_information_April2018.csv
employee_information_April2019.csv
employee_information_April2020.csv
employee_information_April2021.csv


In [87]:
school_df_2019 = pd.read_excel('data/SPR_SY1819_School_Metric_Scores_20200127.xlsx', sheet_name='SPR SY2018-2019')
school_df_2019.sample(10)

,School,SRC School ID,ULCS Code,Report,Rpt Type Long,Street Address,City,State,Zip Code,Phone Number,...,FAFSA Tier,Student Survey College & Career Score,Student Survey College & Career Pts Earn,Student Survey College & Career Pts Poss,Student Survey College & Career Pct Earn,Student Survey College & Career Tier,Teach Effect MMS Dist Score,Teach Effect MMS Prof Score,Teacher Attendance Score,Student Survey Teaching Score
169,Francis Hopkinson School,730,7300,K8,K8 School,4001 L St.,Philadelphia,PA,19124,215-400-3970,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,0,96,62,52
311,Mastery Charter School at Cleveland,X20,3420,K8,K8 School,3701 N. 19th St.,Philadelphia,PA,19140,215-227-5042,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,Data Not Available,0
314,Memphis Street Academy Charter School at J.P. ...,X23,3423,MS,Middle School,2950 Memphis St.,Philadelphia,PA,19134,215-291-4709,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,Data Not Available,60
251,People for People Charter School,W35,3335,K8,K8 School,800 N. Broad St.,Philadelphia,PA,19130,215-763-7060,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Data Not Available,Data Not Available,Data Not Available,52
243,Franklin Towne Charter High School,W31,3331,HS,High School,5301 Tacony St.,Philadelphia,PA,19137,215-289-5000,...,MODEL,35,0.35,1.00,35,WATCH,Data Not Available,Data Not Available,Data Not Available,64
39,Franklin Learning Center,229,2290,HS,High School,616 N. 15th St.,Philadelphia,PA,19130,215-400-7570,...,REINFORCE,29,0.29,1.00,29,WATCH,4,96,50,46
306,Mastery Charter School at Gratz,X16,3416,HS,High School,1798 W. Hunting Park Ave.,Philadelphia,PA,19140,215-227-4408,...,MODEL,0,0.00,1.00,0,INTERVENE,Data Not Available,Data Not Available,Data Not Available,0
172,Henry W. Lawton School,733,7330,ES,Elementary School,6101 Jackson St.,Philadelphia,PA,19135,215-400-3340,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,0,98,46,75
209,Louis H. Farrell School,838,8380,K8,K8 School,8300 Castor Ave.,Philadelphia,PA,19152,215-400-3230,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,3,97,57,54
180,Bayard Taylor School,744,7440,ES,Elementary School,3698 N. Randolph St.,Philadelphia,PA,19140,215-400-3880,...,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,Not Applicable to Report Type,0,97,69,62


In [88]:
school_df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Columns: 337 entries, School to Student Survey Teaching Score
dtypes: float64(6), int64(9), object(322)
memory usage: 858.4+ KB


### Modeling

### Evaluation